# Tests for DataMux

In [1]:
import websockets
import logging
from serializers import Serializer

from typing import List

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
logging.basicConfig(format="%(asctime)-15s %(message)s", level=logging.INFO)
serializer = Serializer("json")

websocket = await websockets.connect('ws://127.0.0.1:3300/ws')

In [8]:
collection_arr: List[dict] = []
async def list_collections():
    sendmsg = serializer.encode(b"list_collections", {})
    await websocket.send(sendmsg)
    recvmsg = await websocket.recv()
    topic, content = serializer.decode(recvmsg)
    print(topic)
    collections = content["collections"]
    print(*collections, sep="\n")
    collection_arr.extend(collections)
await list_collections()

b'list_collections'
{'name': 'adhd_sin', 'description': 'Eye movements of participants during an audiovisual-SIN task', 'keywords': ['SIN', 'ADHD'], 'authors': [{'name': 'Anne M. P. Michalek', 'affiliation': 'Old Dominion University', 'email': 'aperrott@odu.edu'}], 'streams': {'gaze': {'name': 'Gaze', 'description': 'gaze point', 'unit': '', 'frequency': 60.0, 'fields': {'x': {'name': 'gaze x', 'description': 'Normalized X-Gaze Point'}, 'y': {'name': 'gaze y', 'description': 'Normalized Y-Gaze Point'}}, 'index': {'t': {'name': 'timestamp', 'description': 'Recording Timestamp'}}, 'attrs': {}}, 'pupil': {'name': 'Pupil', 'description': 'pupil diameter', 'unit': 'mm', 'frequency': 60.0, 'fields': {'d': {'name': 'pupil d', 'description': 'Pupil Diameter'}}, 'index': {'t': {'name': 'timestamp', 'description': 'Recording Timestamp'}}, 'attrs': {}}}, 'groups': {'participant': {'description': 'The ID of each participant', 'values': ['3', '5', '12', '14', '18', '21', '24', '26', '30', '32', '34

In [9]:
streams_arr: List[dict] = []
async def list_collection_streams():
    collection = collection_arr[0]
    sendmsg = serializer.encode(b"list_collection_streams", dict(collection_name=collection["name"]))
    await websocket.send(sendmsg)
    recvmsg = await websocket.recv()
    topic, content = serializer.decode(recvmsg)
    collection_name = content["collection_name"]
    print(topic, collection_name)
    streams = content["streams"]
    print(*streams, sep="\n")
    streams_arr.extend(streams)
await list_collection_streams()

b'list_collection_streams' adhd_sin
{'name': 'Gaze', 'description': 'gaze point', 'unit': '', 'frequency': 60.0, 'fields': {'x': {'name': 'gaze x', 'description': 'Normalized X-Gaze Point'}, 'y': {'name': 'gaze y', 'description': 'Normalized Y-Gaze Point'}}, 'index': {'t': {'name': 'timestamp', 'description': 'Recording Timestamp'}}, 'attrs': {'diagnosis': 'Non-ADHD', 'gender': 'female', 'noise': '0', 'participant': '12', 'question': '10', 'collection': 'adhd_sin', 'id': 'gaze'}}
{'name': 'Pupil', 'description': 'pupil diameter', 'unit': 'mm', 'frequency': 60.0, 'fields': {'d': {'name': 'pupil d', 'description': 'Pupil Diameter'}}, 'index': {'t': {'name': 'timestamp', 'description': 'Recording Timestamp'}}, 'attrs': {'diagnosis': 'Non-ADHD', 'gender': 'female', 'noise': '0', 'participant': '12', 'question': '10', 'collection': 'adhd_sin', 'id': 'pupil'}}
{'name': 'Gaze', 'description': 'gaze point', 'unit': '', 'frequency': 60.0, 'fields': {'x': {'name': 'gaze x', 'description': 'Norma

In [ ]:
async def restream_streams():
    stream = streams_arr[0]
    sendmsg = serializer.encode(b"restream_collection_stream", dict(collection_name=collection["name"], stream_name=stream["name"]))
    await websocket.send(sendmsg)
    recvmsg = await websocket.recv()
    topic, content = serializer.decode(recvmsg)
    collection_name = content["collection_name"]
    print(topic, collection_name)
    streams = content["streams"]
    print(*streams, sep="\n")
await list_collection_streams()